# Notebook pour l'entraînement et le suivi avec MLFlow

In [1]:
import pandas as pd
import sklearn
import mlflow
import evidently


In [3]:
import numpy as np

# Définir le nombre d'échantillons
num_samples = 1000

# Créer un DataFrame avec des colonnes fictives
data = pd.DataFrame({
    'ID': range(1, num_samples + 1),  # Un identifiant unique pour chaque échantillon
    'AGE': np.random.randint(18, 70, num_samples),  # Âge entre 18 et 70 ans
    'INCOME': np.random.randint(20000, 120000, num_samples),  # Revenu annuel
    'CREDIT_SCORE': np.random.randint(300, 850, num_samples),  # Score de crédit entre 300 et 850
    'LOAN_AMOUNT': np.random.randint(1000, 50000, num_samples),  # Montant du prêt
    'DURATION': np.random.randint(6, 60, num_samples),  # Durée du prêt en mois
    'TARGET': np.random.choice([0, 1], num_samples)  # Cible binaire (0 = non, 1 = oui)
})

# Enregistrer le DataFrame en tant que fichier CSV
data.to_csv("application_train.csv", index=False)

print(data.head())


   ID  AGE  INCOME  CREDIT_SCORE  LOAN_AMOUNT  DURATION  TARGET
0   1   30   53107           381        15694        45       0
1   2   22   41920           574        47094        56       0
2   3   58   99948           497        41705        22       1
3   4   18   59422           656        32160        54       0
4   5   23   65241           495        37032        19       0


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import mlflow
import mlflow.sklearn

# Charger les données
data = pd.read_csv("../models/application_train.csv")
X = data.drop('TARGET', axis=1)
y = data['TARGET']

# Diviser les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser MLFlow
mlflow.set_experiment("Credit Scoring")

# Entraîner le modèle
with mlflow.start_run() as run:
    print("Début de l'entraînement du modèle.")
    
    clf = RandomForestClassifier()
    
    # Utiliser GridSearchCV pour optimiser les hyperparamètres
    param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
    grid_search = GridSearchCV(clf, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    print("Optimisation des hyperparamètres terminée.")
    
    # Prédiction et évaluation
    y_pred = grid_search.predict(X_test)
    auc_score = roc_auc_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques et modèle dans MLFlow
    mlflow.log_param("best_params", grid_search.best_params_)
    mlflow.log_metric("AUC", auc_score)
    mlflow.log_metric("Accuracy", accuracy)
    
    # Enregistrer le modèle
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")
    
    print("Modèle enregistré avec AUC :", auc_score)


Début de l'entraînement du modèle.
Optimisation des hyperparamètres terminée.


2024/10/14 13:00:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré avec AUC : 0.597125916993267


In [7]:
# train_model.ipynb

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
import mlflow
import mlflow.sklearn

# Charger les données
data = pd.read_csv("../models/application_train.csv")
X = data.drop('TARGET', axis=1)
y = data['TARGET']

# Diviser les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser MLFlow
mlflow.set_experiment("Credit Scoring")

# Entraîner le modèle
with mlflow.start_run():
    clf = RandomForestClassifier()
    
    # Utiliser GridSearchCV pour optimiser les hyperparamètres
    param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
    grid_search = GridSearchCV(clf, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    
    # Prédiction et évaluation
    y_pred = grid_search.predict(X_test)
    auc_score = roc_auc_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    # Log des métriques et modèle dans MLFlow
    mlflow.log_param("best_params", grid_search.best_params_)
    mlflow.log_metric("AUC", auc_score)
    mlflow.log_metric("Accuracy", accuracy)
    
    # Enregistrer le modèle
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")
    
    print("Modèle enregistré avec AUC :", auc_score)


2024/10/14 11:28:23 INFO mlflow.tracking.fluent: Experiment with name 'Credit Scoring' does not exist. Creating a new experiment.
2024/10/14 11:28:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modèle enregistré avec AUC : 0.5777308813184605
